# YoLo

#1. YoLo核心思想
- YOLO的核心思想就是利用整张图作为网络的输入，直接在输出层回归bounding box的位置和bounding box所属的类别。
- 没记错的话faster RCNN中也直接用整张图作为输入，但是faster-RCNN整体还是采用了RCNN那种 proposal+classifier的思想，只不过是将提取proposal的步骤放在CNN中实现了。

#2. YoLo实现方法
- 将一幅图像分成SxS(7*7)个网格(grid cell)，如果某个object的中心 落在这个网格中，则这个网格就负责预测这个object
- 每个网格要预测B个bounding box，每个bounding box除了要回归自身的位置之外，还要附带预测一个confidence值。这个confidence代表了所预测的box中含有object的置信度和这个box预测的有多准两重信息，其值是这样计算的$P_r(Object) * IOU_{pred}^{truth}$，其中如果有object落在一个grid cell里面，那么第一项取1，否则取0；第二项是预测的bounding box和实际的ground truth之间的IoU的值
- 每个bounding box要预测(x, y, w, h)和confidence共5个值，每个网格还要预测一个类别信息，记为C类。则SxS个网格，每个网格要预测B个bounding box还要预测C个categories。输出就是S x S x (5*B+C)的一个tensor。注意：class信息是针对每个网格的，confidence信息是针对每个bounding box的。
- 在test的时候，每个网格预测的class信息和bounding box预测的confidence信息相乘，就得到每个bounding box的class-specific confidence score: $P_r(Class_i|Object) * P_r(Object) * IOU_{pred}^{truth}=P_r(Class_i) * IOU_{pred}^{truth}$.等式左边第一项就是每个网格预测的类别信息，第二三项就是每个bounding box预测的confidence。这个乘积即encode了预测的box属于某一类的概率，也有该box准确度的信息。
- 得到每个box的class-specific confidence score以后，设置阈值，滤掉得分低的boxes，对保留的boxes进行NMS处理，就得到最终的检测结果。


#3. 举例说明
在PASCAL VOC中，图像输入为448x448，取S=7，B=2，一共有20个类别(C=20)。则输出就是7x7x30的一个tensor。整个网络结构如下图所示![images](images/02.png)

#4 YoLo的实现细节
- 每个grid有30维，这30维中，8维是回归box的坐标，2维是box的confidence，还有20维是类别,其中坐标的x,y用对应网格的offset归一化到0-1之间，w,h用图像的width和height归一化到0-1之间
- 在实现中，最主要的就是怎么设计损失函数，让这个三个方面得到很好的平衡。作者简单粗暴的全部采用了sum-squared error loss来做这件事。但是这样做存在几个问题： 
   - 8维的localization error和20维的classification error同等重要显然是不合理的； 
   - 如果一个网格中没有object（一幅图中这种网格很多），那么就会将这些网格中的box的confidence push到0，相比于较少的有object的网格，这种做法是overpowering的，这会导致网络不稳定甚至发散。 
- 针对上述问题，解决办法是：
   - 更重视8维的坐标预测，给这些损失前面赋予更大的loss weight, 记为在pascal VOC训练中取5。
   - 对没有object的box的confidence loss，赋予小的loss weight，记为在pascal VOC训练中取0.5。
   - 有object的box的confidence loss和类别的loss的loss weight正常取1
- 对不同大小的box预测中，相比于大box预测偏一点，小box预测偏一点肯定更不能被忍受的。而sum-square error loss中对同样的偏移loss是一样。 为了缓和这个问题，作者用了一个比较取巧的办法，就是将box的width和height取平方根代替原本的height和width。这个参考下面的图很容易理解，小box的横轴值较小，发生偏移时，反应到y轴上相比大box要大
- 一个网格预测多个box，希望的是每个box predictor专门负责预测某个object。具体做法就是看当前预测的box与ground truth box中哪个IoU大，就负责哪个。这种做法称作box predictor的specialization

#5. 损失函数
![images](images/03.png)
这个损失函数中： 
- 只有当某个网格中有object的时候才对classification error进行惩罚。
- 只有当某个box predictor对某个ground truth box负责的时候，才会对box的coordinate error进行惩罚，而对哪个ground truth box负责就看其预测值和ground truth box的IoU是不是在那个cell的所有box中最大。